## Inputs

In [110]:
user_preferences = {
  "preferredStudyTimes": ["late morning", "afternoon", "night"],
  "preferredSessionDuration": {
    "min": 30,
    "max": 90
  },
  "revisionFrequency": "2-3 reviews per topic",
  "breakDuration": 15
}

availability = {
    "12/02/2025": ["09:00-11:00", "14:00-16:00", "20:00-22:00"],
    "14/02/2025": ["10:00-12:00", "14:00-18:00", "20:00-22:00"],
    "15/02/2025": ["08:00-11:00", "14:00-16:00", "20:00-22:00"],
    "17/02/2025": ["10:00-12:00", "14:00-17:00", "20:00-22:00"]
}

courses = [
    {
        "name": "Operating Systems",
        "credit": 3,
        "examDate": "25/02/2025",
        "examTime": "09:00–11:00",
        "topics": [
            {"title": "CPU Scheduling", "difficulty": 3, "confidence": 2, "studyTime": 90},
            {"title": "Deadlocks", "difficulty": 4, "confidence": 3, "studyTime": 75},
            {"title": "Virtual Memory", "difficulty": 5, "confidence": 2, "studyTime": 120}
        ]
    },
    {
        "name": "Data Structures and Algorithms",
        "credit": 3,
        "examDate": "22/02/2025",
        "examTime": "10:00–12:00",
        "topics": [
            {"title": "Trees & Graphs", "difficulty": 3, "confidence": 2, "studyTime": 80},
            {"title": "Sorting Algorithms", "difficulty": 2, "confidence": 4, "studyTime": 50},
            {"title": "Dynamic Programming", "difficulty": 5, "confidence": 2, "studyTime": 110}
        ]
    },
    {
        "name": "Machine Learning",
        "credit": 3,
        "examDate": "23/02/2025",
        "examTime": "13:00–15:00",
        "topics": [
            {"title": "Regression Models", "difficulty": 3, "confidence": 3, "studyTime": 70},
            {"title": "Classification", "difficulty": 4, "confidence": 2, "studyTime": 90},
            {"title": "Neural Networks", "difficulty": 5, "confidence": 2, "studyTime": 130}
        ]
    },
    {
        "name": "Database Systems",
        "credit": 2,
        "examDate": "27/02/2025",
        "examTime": "09:30–11:30",
        "topics": [
            {"title": "SQL Joins", "difficulty": 2, "confidence": 4, "studyTime": 40},
            {"title": "Normalization", "difficulty": 3, "confidence": 3, "studyTime": 60},
            {"title": "Indexing & Query Optimization", "difficulty": 4, "confidence": 2, "studyTime": 90}
        ]
    },
    {
        "name": "Computer Networks",
        "credit": 2,
        "examDate": "25/02/2025",
        "examTime": "14:00–16:00",
        "topics": [
            {"title": "TCP/IP Model", "difficulty": 2, "confidence": 4, "studyTime": 50},
            {"title": "Routing Protocols", "difficulty": 4, "confidence": 2, "studyTime": 100},
            {"title": "Congestion Control", "difficulty": 3, "confidence": 3, "studyTime": 60}
        ]
    }
]

assignments = [
    {
        "course": "Computer Networks",
        "title": "Midterm Essay",
        "associatedTopic": ["TCP/IP Model"],
        "dueDate": "14/02/2025",
        "time": "15.00",
        "estimatedTime": 50 
    },
    {
        "course": "Machine Learning",
        "title": "Final Project",
        "assiociatedTopic": ["Regression Models", "Classification", "Neural Networks"],
        "dueDate": "10/03/2025",
        "estimatedTime": 180
    }
]

## Helpers

In [111]:
import math

def round10(x):
    return math.ceil(x / 10) * 10

### Handle The sessions

In [112]:
def estimate_study_time(difficulty, confidence, base_time=None):
    """
    Estimate adjusted study time based on difficulty and confidencial.
    Rounds the result to the nearest 10 minutes.
    """
    if base_time is None:
        base_time = 60  # default if not provided

    adjustment = (difficulty - confidence) * 0.1
    adjusted_time = base_time * (1 + adjustment)

    # Clamp to minimum of 15 minutes, then round to nearest 10
    adjusted_time = max(15, adjusted_time)
    rounded_time = round(adjusted_time / 10) * 10

    return rounded_time

In [113]:
def estimate_sessions(adjusted_study_time, revision_freq, preferred_study_duration):
    """
    Break adjusted study time into sessions based on revision strategy.
    Enforce preferred session duration by splitting oversized chunks.
    All durations are rounded UP to the nearest 10 minutes.
    """

    min_dur = preferred_study_duration["min"]
    max_dur = preferred_study_duration["max"]

    sessions = []

    if revision_freq == "single deep review before exam":
        deep = round10(adjusted_study_time)
        review = round10(adjusted_study_time * 0.2)
        sessions = [deep, review]

    elif revision_freq == "2-3 reviews per topic":
        overview = round10(adjusted_study_time * 0.2)
        deep = round10(adjusted_study_time)
        review = round10(adjusted_study_time * 0.3)
        sessions = [overview, deep, review]

    elif revision_freq == "daily review sessions":
        review = round10(adjusted_study_time * 0.25)
        core_time = adjusted_study_time

        # 🗓️ Target up to 5 daily sessions (or as many fit given min_dur)
        num_daily = min(5, max(1, core_time // min_dur))

        base = core_time // num_daily
        remainder = core_time % num_daily

        sessions = []
        for i in range(num_daily):
            session_length = base + (1 if i < remainder else 0)
            sessions.append(round10(session_length))

        sessions.append(review)
        
    else: # Default fallback if revision_freq is not recognized
        core = round10(adjusted_study_time)
        review = round10(adjusted_study_time * 0.2) # Default to a simple core + review
        sessions = [core, review]


    # 💡 Enforce preferred max duration (split if too long)
    final_sessions = []
    for s in sessions:
        if s > max_dur:
            parts = math.ceil(s / max_dur)
            per_part = round10(s / parts)
            final_sessions.extend([per_part] * parts)
        elif s < min_dur and s > 0: # Ensure session is not too short, unless it's 0
            final_sessions.append(min_dur) # Or round10(s) if very short sessions are acceptable
        elif s >= min_dur:
            final_sessions.append(s)
        # if s is 0, it will be ignored

    return final_sessions

In [114]:
def create_assignment_sessions(estimated_time, preferred_study_duration):
    """
    Transforms an assignment's estimated time into study sessions,
    respecting the preferred maximum session duration.
    All durations are rounded UP to the nearest 10 minutes.
    """
    def round10(x):
        return math.ceil(x / 10) * 10

    min_dur = preferred_study_duration["min"]
    max_dur = preferred_study_duration["max"]
    
    assignment_sessions = []
    
    if estimated_time == 0:
        return []

    if estimated_time > max_dur:
        parts = math.ceil(estimated_time / max_dur)
        per_part = round10(estimated_time / parts)
        assignment_sessions.extend([per_part] * parts)
    else:
        assignment_sessions.append(round10(estimated_time))
        
    # Ensure no session is too short if it got split that way,
    # although with round10(estimated_time / parts) it's less likely
    # to be below min_dur unless estimated_time is very close to max_dur
    # or parts is large.
    final_assignment_sessions = []
    for s_val in assignment_sessions:
        if s_val < min_dur and s_val > 0 : # if a session ends up less than min_dur, bump to min_dur
            final_assignment_sessions.append(min_dur)
        else:
            final_assignment_sessions.append(s_val)

    return final_assignment_sessions

#### Session Results

In [115]:
print("📚 Study Sessions for Courses:")
for course in courses:
    print(f"\n📘 Course: {course['name']}")
    for topic in course["topics"]:
        base_time = topic.get("studyTime", 60)
        adjusted_time = estimate_study_time(topic["difficulty"], topic["confidence"], base_time)
        sessions = estimate_sessions(adjusted_time, user_preferences["revisionFrequency"], user_preferences["preferredSessionDuration"])
        session_info = ", ".join([f"{int(s)} min" for s in sessions if s > 0])
        if session_info: # only print if there are sessions
            print(f"  • {topic['title']} → Total Adjusted: {adjusted_time} min → Sessions: {session_info}")
        else:
            print(f"  • {topic['title']} → Total Adjusted: {adjusted_time} min → No sessions generated (check duration/logic)")
print("\n" + "="*30 + "\n")
print("📝 Assignment Sessions:")
for assignment in assignments:
    print(f"\n📌 Assignment: {assignment['title']} ({assignment['course']})")
    estimated_time = assignment["estimatedTime"]
    sessions = create_assignment_sessions(estimated_time, user_preferences["preferredSessionDuration"])
    session_info = ", ".join([f"{int(s)} min" for s in sessions if s > 0])
    if session_info: # only print if there are sessions
        print(f"  • Estimated Time: {estimated_time} min → Sessions: {session_info}")
    else:
        print(f"  • Estimated Time: {estimated_time} min → No sessions generated (check duration/logic)")

📚 Study Sessions for Courses:

📘 Course: Operating Systems
  • CPU Scheduling → Total Adjusted: 100 min → Sessions: 30 min, 50 min, 50 min, 30 min
  • Deadlocks → Total Adjusted: 80 min → Sessions: 30 min, 80 min, 30 min
  • Virtual Memory → Total Adjusted: 160 min → Sessions: 40 min, 80 min, 80 min, 50 min

📘 Course: Data Structures and Algorithms
  • Trees & Graphs → Total Adjusted: 90 min → Sessions: 30 min, 90 min, 30 min
  • Sorting Algorithms → Total Adjusted: 40 min → Sessions: 30 min, 40 min, 30 min
  • Dynamic Programming → Total Adjusted: 140 min → Sessions: 30 min, 70 min, 70 min, 50 min

📘 Course: Machine Learning
  • Regression Models → Total Adjusted: 70 min → Sessions: 30 min, 70 min, 30 min
  • Classification → Total Adjusted: 110 min → Sessions: 30 min, 60 min, 60 min, 40 min
  • Neural Networks → Total Adjusted: 170 min → Sessions: 40 min, 90 min, 90 min, 60 min

📘 Course: Database Systems
  • SQL Joins → Total Adjusted: 30 min → Sessions: 30 min, 30 min, 30 min
  • N

### Handle Scheduling

In [116]:
def get_preferred_time_ranges(pref_strings):
    time_periods_map = {
        "early morning": (4, 8), "late morning": (8, 12), "afternoon": (12, 18),
        "evening": (18, 22), "night": (22, 24), "late night": (0, 4)
    }
    return [time_periods_map[p] for p in pref_strings if p in time_periods_map]

In [117]:
def get_session_types_for_freq(revision_freq):
    """
    Maps revision frequency to a list of session types.
    Ensures we avoid division by zero in session generation.
    """
    mapping = {
        "single deep review before exam": ["deep", "review"],
        "2-3 reviews per topic": ["overview", "deep", "review"],
        "daily review sessions": ["deep"] * 5 + ["review"],
    }
    return mapping.get(revision_freq, ["deep", "review"])  # default fallback


In [118]:
from datetime import datetime, timedelta

def parse_timeblock(tblock_str):
    start_str, end_str = tblock_str.split("-")
    return (datetime.strptime(start_str.strip(), "%H:%M"),
            datetime.strptime(end_str.strip(), "%H:%M"))

def timeblock_duration(start, end):
    return int((end - start).total_seconds() / 60)

def format_time(dt):
    return dt.strftime("%H:%M")

def split_timeblock(start, duration):
    """Returns (block_start, block_end) after consuming `duration`."""
    end = start + timedelta(minutes=duration)
    return (start, end)

In [ ]:
def build_all_sessions(courses, assignments, prefs):
    session_list = []
    
    for course in courses:
        for topic in course["topics"]:
            base = topic["studyTime"]
            adjusted = estimate_study_time(topic["difficulty"], topic["confidence"], base)
            session_durations = estimate_sessions(adjusted, prefs["revisionFrequency"], prefs["preferredSessionDuration"])
            types = get_session_types_for_freq(prefs["revisionFrequency"])
            for i, duration in enumerate(session_durations):
                session_list.append({
                    "course": course["name"],
                    "topic": topic["title"],
                    "type": types[i % len(types)],
                    "duration": duration,
                    "due": datetime.strptime(course["examDate"], "%d/%m/%Y"),
                    "priority": 0 if types[i % len(types)] == "review" else 1  # reviews go last
                })

    for assign in assignments:
        sessions = create_assignment_sessions(assign["estimatedTime"], prefs["preferredSessionDuration"])
        for duration in sessions:
            session_list.append({
                "course": assign["course"],
                "topic": ", ".join(assign.get("associatedTopic", assign.get("assiociatedTopic", []))),
                "type": "assignment",
                "duration": duration,
                "due": datetime.strptime(assign["dueDate"], "%d/%m/%Y"),
                "priority": 1
            })
    
    # Sort by due date, then course name, then topic
    return sorted(session_list, key=lambda s: (s["due"], s["course"], s["topic"], s["priority"]))

In [120]:
def schedule_review_sessions(sessions, availability, prefs):
    scheduled = []
    review_sessions = [s for s in sessions if s["type"] == "review"]
    sessions_remaining = [s for s in sessions if s["type"] != "review"]
    break_min = prefs["breakDuration"]

    for review in review_sessions:
        review_day = (review["due"] - timedelta(days=1)).strftime("%d/%m/%Y")
        if review_day not in availability:
            continue

        for i, block in enumerate(availability[review_day]):
            start, end = parse_timeblock(block)
            duration = timeblock_duration(start, end)
            if duration >= review["duration"]:
                sess_start, sess_end = split_timeblock(start, review["duration"])
                break_end = sess_end + timedelta(minutes=break_min)

                scheduled.append({
                    "date": review_day,
                    "start": format_time(sess_start),
                    "end": format_time(sess_end),
                    "course": review["course"],
                    "topic": review["topic"],
                    "type": review["type"]
                })

                # Adjust remaining availability
                if break_end < end:
                    availability[review_day][i] = f"{format_time(break_end)}-{format_time(end)}"
                else:
                    availability[review_day].pop(i)
                break  # schedule one session per course only
    return scheduled, sessions_remaining


In [121]:
def schedule_sessions(session_list, availability, prefs, already_scheduled=[]):
    scheduled = already_scheduled[:]
    break_min = prefs["breakDuration"]

    for session in session_list:
        scheduled_flag = False
        for date_str in sorted(availability.keys()):
            date_obj = datetime.strptime(date_str, "%d/%m/%Y")
            if date_obj > session["due"]:
                continue

            for i, block in enumerate(availability[date_str]):
                start, end = parse_timeblock(block)
                duration = timeblock_duration(start, end)
                if duration >= session["duration"]:
                    sess_start, sess_end = split_timeblock(start, session["duration"])
                    break_end = sess_end + timedelta(minutes=break_min)

                    scheduled.append({
                        "date": date_str,
                        "start": format_time(sess_start),
                        "end": format_time(sess_end),
                        "course": session["course"],
                        "topic": session["topic"],
                        "type": session["type"]
                    })

                    # Update availability
                    if break_end < end:
                        availability[date_str][i] = f"{format_time(break_end)}-{format_time(end)}"
                    else:
                        availability[date_str].pop(i)
                    scheduled_flag = True
                    break
            if scheduled_flag:
                break
    return scheduled

In [122]:
from collections import defaultdict

# Build all sessions
all_sessions = build_all_sessions(courses, assignments, user_preferences)

# Reserve and schedule all review sessions 1 day before exam
review_scheduled, remaining_sessions = schedule_review_sessions(all_sessions, availability, user_preferences)

# Schedule remaining sessions (study + assignments)
fully_scheduled = schedule_sessions(remaining_sessions, availability, user_preferences, already_scheduled=review_scheduled)

# Group scheduled sessions by date
scheduled_by_date = defaultdict(list)
scheduled_topics_set = set()

for sess in fully_scheduled:
    key = f"{sess['date']}"
    scheduled_by_date[key].append(sess)
    # Track unique (topic, type, course) for unscheduled detection
    scheduled_topics_set.add((sess["topic"], sess["type"], sess["course"]))

# 📅 Study Plan
print("\n\n📅 Study Plan:\n")
for date in sorted(scheduled_by_date.keys(), key=lambda x: datetime.strptime(x, "%d/%m/%Y")):
    print(f"{date}")
    for sess in sorted(scheduled_by_date[date], key=lambda x: x["start"]):
        print(f"  {sess['start']} - {sess['end']} ➔ {sess['course']} - {sess['topic']} ({sess['type'].capitalize()})")

# ⚠️ Unscheduled Sessions
unscheduled = []
for s in all_sessions:
    if (s["topic"], s["type"], s["course"]) not in scheduled_topics_set:
        unscheduled.append(s)

if unscheduled:
    print("\n\n⚠️ Unscheduled Sessions:")
    for u in unscheduled:
        due_str = u["due"].strftime("%d/%m/%Y")
        print(f"  ❌ {u['course']} - {u['topic']} ({u['type'].capitalize()}, {u['duration']} min) → Due: {due_str}")
else:
    print("\n✅ All sessions scheduled successfully!")

# 📆 Exam Dates
print("\n📆 Exam Dates:")
for course in courses:
    print(f"  📝 {course['name']}: {course['examDate']} ({course['examTime']})")

# 📌 Assignment Deadlines
print("\n📌 Assignment Deadlines:")
for a in assignments:
    title = a["title"]
    course = a["course"]
    due = a["dueDate"]
    print(f"  📎 {course} - {title}: Due {due}")



📅 Study Plan:

12/02/2025
  09:00 - 09:50 ➔ Computer Networks - TCP/IP Model (Assignment)
  10:05 - 10:35 ➔ Data Structures and Algorithms - Dynamic Programming (Overview)
  14:00 - 15:10 ➔ Data Structures and Algorithms - Dynamic Programming (Deep)
  15:25 - 15:55 ➔ Data Structures and Algorithms - Sorting Algorithms (Overview)
  20:00 - 20:50 ➔ Data Structures and Algorithms - Dynamic Programming (Overview)
  21:05 - 21:45 ➔ Data Structures and Algorithms - Sorting Algorithms (Deep)
14/02/2025
  10:00 - 10:30 ➔ Data Structures and Algorithms - Trees & Graphs (Overview)
  10:45 - 11:15 ➔ Machine Learning - Classification (Overview)
  11:30 - 12:00 ➔ Machine Learning - Regression Models (Overview)
  14:00 - 15:30 ➔ Data Structures and Algorithms - Trees & Graphs (Deep)
  15:45 - 16:45 ➔ Machine Learning - Classification (Deep)
  17:00 - 17:40 ➔ Machine Learning - Classification (Overview)
  20:00 - 20:40 ➔ Machine Learning - Neural Networks (Overview)
  20:55 - 21:55 ➔ Machine Learni

In [123]:
from datetime import datetime, timedelta

def parse_time(time_str):
    """Parse time string 'HH:MM' to datetime.time."""
    return datetime.strptime(time_str, "%H:%M").time()

def slot_duration_minutes(start, end):
    """Calculate duration in minutes between two time strings."""
    start_dt = datetime.strptime(start, "%H:%M")
    end_dt = datetime.strptime(end, "%H:%M")
    return int((end_dt - start_dt).total_seconds() // 60)

def csp_schedule_sessions(sessions, availability_slots):
    """
    Map study sessions to available time slots using a CSP-like greedy approach.
    Returns a list of assignments: [{'date', 'start', 'end', 'session_duration'}]
    """
    assignments = []
    session_idx = 0

    for slot in availability_slots:
        slot_start = datetime.strptime(f"{slot['date']} {slot['start']}", "%d/%m/%Y %H:%M")
        slot_end = datetime.strptime(f"{slot['date']} {slot['end']}", "%d/%m/%Y %H:%M")
        slot_minutes = int((slot_end - slot_start).total_seconds() // 60)

        while session_idx < len(sessions) and slot_minutes >= sessions[session_idx]:
            session_duration = sessions[session_idx]
            session_start = slot_start
            session_end = session_start + timedelta(minutes=session_duration)
            assignments.append({
                'date': slot['date'],
                'start': session_start.strftime("%H:%M"),
                'end': session_end.strftime("%H:%M"),
                'session_duration': session_duration
            })
            slot_start = session_end
            slot_minutes -= session_duration
            session_idx += 1

        if session_idx >= len(sessions):
            break

    return assignments

# Prepare availability slots
availability_slots = transform_availability(availability)

# Example: map sessions for the last calculated 'sessions' variable
mapped_sessions = csp_schedule_sessions(sessions, availability_slots)

for a in mapped_sessions:
    print(f"{a['date']} | {a['start']}-{a['end']} → {a['session_duration']} min")